## Retrieve parsed collection

In [1]:
import pandas as pd
import numpy as np
from gensim import models, similarities
from tqdm import tqdm
import spacy
import joblib
from modules.db import mongo_base
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.feature_extraction import FeatureHasher, DictVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from modules.utils.CustomTwokenizer import CustomTwokenizer

### Store feature collection as dataframe

In [12]:
# connection_params = ["twitter", "crowdflower_features"]
# client = mongo_base.connect()
# db_name = connection_params[0]
# connection_params.insert(0, client)

# query = {}
# query["filter"] = {}
# query["projection"] = {}
# query["limit"] = 0
# query["skip"] = 0
# query["no_cursor_timeout"] = True
# cursor = mongo_base.finder(connection_params, query, False)
# df = pd.DataFrame(list(cursor))
# df.head()

#### Merge dataframes

In [13]:
# connection_params = ["twitter", "crowdflower_features_emo"]
# client = mongo_base.connect()
# db_name = connection_params[0]
# connection_params.insert(0, client)

# query = {}
# query["filter"] = {}
# query["projection"] = {"emotions":1}
# query["limit"] = 0
# query["skip"] = 0
# query["no_cursor_timeout"] = True
# cursor = mongo_base.finder(connection_params, query, False)
# df_emo = pd.DataFrame(list(cursor))
# df = pd.DataFrame.merge(df, df_emo, on="_id")
# df.head()

#### Pickle the raw feature collection

In [2]:
spacy_en_model = "en_core_web_md"
spacy_glove_model = "en_vectors_glove_md"
crowdflower_persistence_raw = 'data/persistence/df/crowdflower_features_raw.pkl.compressed'
crowdflower_persistence = 'data/persistence/df/crowdflower_features.pkl.compressed'
nlp = spacy.load(spacy_en_model, create_make_doc=CustomTwokenizer)
# joblib.dump(df, crowdflower_persistence_raw, compress=True)

In [3]:
# df = joblib.load(crowdflower_persistence_raw)

### Create a dataframe with classifier features

In [4]:
# feat_df = df[['_id', 'text', 'annotation_label', 'hs_keyword_matches', 'hs_keyword_count', 'unknown_words', 'unknown_words_count', 'comment_length', 'brown_cluster_ids', 'feat_dependency_contexts', 'feat_word_dep_root', 'feat_pos_dep_rootPos', 'feat_word_root_rootparent', 'feat_dep_unigrams', 'feat_dep_bigrams', 'feat_dep_trigrams']]

In [3]:
# joblib.dump(feat_df, crowdflower_persistence, compress=True)
feat_df = joblib.load(crowdflower_persistence)
feat_df.head()

,_id,text,annotation_label,hs_keyword_matches,hs_keyword_count,unknown_words,unknown_words_count,comment_length,brown_cluster_ids,feat_dependency_contexts,feat_word_dep_root,feat_pos_dep_rootPos,feat_word_root_rootparent,feat_dep_unigrams,feat_dep_bigrams,feat_dep_trigrams
0,58c659be6541913eb7f119dd,Warning : penny boards will make you a faggot,not_offensive,[faggot],1,[],0,9,"[966, 228, 989, 333, 442, 4618, 602, 19]","[warning_:_punct, warning_make_acl, penny_boar...","[warning_ROOT_warning, penny_compound_boards, ...","[NN_ROOT_NN, NN_compound_NNS, NNS_nsubj_VB, MD...","[warning_warning_warning, penny_boards_make, b...","[warning_warning_ROOT_NN, penny_boards_compoun...",[warning_warning_ROOT_NN | penny_boards_compou...,[warning_warning_ROOT_NN | penny_boards_compou...
1,58c659be6541913eb7f119de,Fuck dykes,hatespeech,[],0,[],0,2,[],"[fuck_dykes_compoundINV, dykes_fuck_compound]","[fuck_compound_dykes, dykes_ROOT_dykes]","[NNP_compound_VBZ, VBZ_ROOT_VBZ]","[fuck_dykes_dykes, dykes_dykes_dykes]","[fuck_dykes_compound_NNP, dykes_dykes_ROOT_VBZ]",[fuck_dykes_compound_NNP | dykes_dykes_ROOT_VBZ],[]
2,58c659be6541913eb7f119df,user_mention user_mention user_mention user_me...,hatespeech,[faggot],1,[jefree],1,14,"[124, 3690, 966, 2442, 1684, 86]","[user_mention_user_mention_compoundINV, user_m...","[user_mention_compound_user_mention, user_ment...","[NN_compound_NN, NN_compound_NN, NN_compound_N...","[user_mention_user_mention_user_mention, user_...","[user_mention_user_mention_compound_NN, user_m...",[user_mention_user_mention_compound_NN | user_...,[user_mention_user_mention_compound_NN | user_...
3,58c659be6541913eb7f119e0,""" user_mention : "" user_mention : user_mention...",hatespeech,[fag],1,[neeeee],1,15,"[228, 228, 1214, 19, 28650, 1014, 981]","[user_mention_""_punct, user_mention_:_punct, u...","[user_mention_ROOT_user_mention, user_mention_...","[NN_ROOT_NN, NN_appos_NN, NN_nsubj_VBZ, VBZ_ac...","[user_mention_user_mention_user_mention, user_...","[user_mention_user_mention_ROOT_NN, user_menti...",[user_mention_user_mention_ROOT_NN | user_ment...,[user_mention_user_mention_ROOT_NN | user_ment...
4,58c659be6541913eb7f119e1,user_mention You heard me bitch but any way I'...,not_offensive,[bitch],1,[nigga],1,20,"[858, 26282, 1898, 2485, 148, 12266, 1349, 753...","[user_mention_heard_relcl, user_mention_bitch_...","[user_mention_ROOT_user_mention, you_nsubj_hea...","[NN_ROOT_NN, PRP_nsubj_VBD, VBD_relcl_NN, PRP_...","[user_mention_user_mention_user_mention, you_h...","[user_mention_user_mention_ROOT_NN, you_heard_...",[user_mention_user_mention_ROOT_NN | you_heard...,[user_mention_user_mention_ROOT_NN | you_heard...


### Load and test dependency2Vec model

In [28]:
model = models.KeyedVectors.load_word2vec_format('data/persistence/df/dim200vecs_hs_candidates_exp6', binary=False)
def most_similar(word, n):
    queries = [w for w in word.vocab if not (word.is_oov or word.is_punct or word.like_num or word.is_stop or word.lower_ == "rt") and w.has_vector and w.lower_ != word.lower_ and w.is_lower == word.is_lower and w.prob >= -15]
    by_similarity = sorted(queries, key=lambda w: word.similarity(w), reverse=True)
    cosine_score = [word.similarity(w) for w in by_similarity]
    return by_similarity[:n], cosine_score[:n]

In [40]:
model.similar_by_word("bitch", topn=20, restrict_vocab=None)

[('bih', 0.8150119781494141),
 ('bihh', 0.7981271743774414),
 ('nicca', 0.783886194229126),
 ('nigglet', 0.7746115922927856),
 ('niglet', 0.7723572254180908),
 ('bish', 0.7560478448867798),
 ('shorty', 0.7535991668701172),
 ('btch', 0.7485767602920532),
 ('nigha', 0.7435488104820251),
 ('kuz', 0.7415286302566528),
 ('chick', 0.7407236099243164),
 ('niggah', 0.7359271049499512),
 ('buh', 0.7343946695327759),
 ('spears', 0.7337879538536072),
 ('nigguh', 0.7332638502120972),
 ('fuckboy', 0.7324934601783752),
 ('bint', 0.7315614819526672),
 ('niggar', 0.7313817739486694),
 ('jiggaboo', 0.7312814593315125),
 ('gook', 0.7270585894584656)]

#### Compare output to standard word2vec model

In [41]:
word = nlp.vocab[u'bitch']
gloVe_result = most_similar(word, 20)
for res in zip(gloVe_result[0], gloVe_result[1]):
    print((res[0].lower_, res[1]))

('bitches', 0.86865010008690136)
('ass', 0.82214490212007696)
('fuck', 0.8190275073922777)
('fucker', 0.7869555594649309)
('fucking', 0.76995593539211227)
('whore', 0.76224389909833756)
('slut', 0.75505237485006749)
('fuckin', 0.75232905563296448)
('asshole', 0.74672725629683701)
('dick', 0.74224040821416348)
('shit', 0.73805058822422542)
('fucked', 0.72900084327361248)
('suck', 0.70949127926097444)
('chick', 0.70751225632834569)
('cunt', 0.70370280187626522)
('fuckers', 0.6952213116761421)
('motherfucker', 0.68982780300108659)
('fucks', 0.68973827279171707)
('pussy', 0.68905374236115746)
('faggot', 0.68810587898398012)


In [5]:
df[0:1]['dep_bigrams'][0][0]
hasher = FeatureHasher(input_type='string', non_negative=True)
v = DictVectorizer(sparse=False)
# raw_X = hasher.transform(df[0:1]['brown_cluster_ids'])
df[0:1]['bigrams'][0][0]
D = [{'bigram': 'Warning penny'}, {'bigram': 'penny boards'}, {'bigram': 'boards will'}, {'bigram': 'boards will'}]
trans = v.fit_transform(D)
v.inverse_transform(trans)
v.get_feature_names()
# raw_X.toarray()
# print(df[0:1]['bigrams'][0])
df[0:1]

,_id,annotation_label,avg_token_length,bigrams,brown_cluster_ids,char_pentagrams,char_quadgrams,char_trigrams,comment_length,conllFormat,...,noun_chunks,pos_dep_rootPos,text,tokens,trigrams,unknown_words,unknown_words_count,uppercase_token_count,word_dep_root,word_root_preRoot
0,58c659be6541913eb7f119dd,not_offensive,4.0,"[Warning penny, penny boards, boards will, wil...","[966, 228, 989, 333, 442, 4618, 602, 19]","[penny, warni, arnin, aggot, board, rning, fag...","[aggo, warn, arni, penn, oard, ards, enny, nin...","[pen, war, nin, mak, enn, you, agg, ggo, fag, ...",9,"[1 warning warning NOUN NN _ 0 ROOT 0:ROOT _, ...",...,"[{'text': 'warning', 'root': 'warning'}, {'tex...","[{'pos': 'NN', 'dep': 'ROOT', 'rootPos': 'NN'}...",Warning : penny boards will make you a faggot,"[warning, penny, boards, faggot]","[Warning penny boards, penny boards will, boar...",[],0,0,"[{'word': 'warning', 'dep': 'ROOT', 'root': 'w...","[{'word': 'warning', 'preRoot': 'warning', 'ro..."


### Setup data split

In [7]:
# let's pick the same random 10% of the data to train with
train_test_set = feat_df.sample(n=int(len(feat_df) / 2), random_state=1965)

X = train_test_set['text']
y = train_test_set['annotation_label']

In [27]:
print("Predicting the labels of the test set...")
print("%d documents" % len(X))
print("%d categories" % len(y.value_counts()))

Predicting the labels of the test set...
7254 documents
2 categories


### Setup generic model experiment

In [8]:
def run_experiment(X, y, pipeline, process_name, num_expts=1):
    scores = list()
    for i in tqdm(range(num_expts)):
        X_train, X_test, y_train, y_test = train_test_split(X, y)
        model = pipeline.fit(X_train, y_train)  # train the classifier
        y_prediction = model.predict(X_test)          # apply the model to the test data
        report = classification_report(y_test, y_prediction)
        score = accuracy_score(y_prediction, y_test)  # compare the results to the gold standard
        scores.append(score)
        print("Classification Report: " + process_name)
        print(report)
        cm = confusion_matrix(y_test, y_prediction)
        print("Confusion matrix:")
        print(cm)
    print(sum(scores) / num_expts)

### Setup naive baseline classification (countVectorizer)

In [9]:
from sklearn.feature_selection import SelectKBest, chi2
# our two ingredients: the ngram counter and the classifier
nm = 5000
vect = CountVectorizer(ngram_range=(3,5), analyzer='char')
clf = LinearSVC()
ch2 = SelectKBest(chi2, k=nm)

# There are just two steps to our process: extracting the ngrams and
# putting them through the classifier. So our Pipeline looks like this:

count_pipeline = Pipeline([
    ('vect', vect),  # extract ngrams from tweet text
    ('kBest', ch2),
    ('clf' , clf),   # feed the output through a classifier
])

# Run the classifcation
run_experiment(X, y, count_pipeline, "CountVectorizer")

100%|██████████| 1/1 [00:03<00:00,  3.88s/it]

Classification Report: CountVectorizer
               precision    recall  f1-score   support

   hatespeech       0.48      0.34      0.40       308
not_offensive       0.87      0.92      0.90      1506

  avg / total       0.81      0.83      0.81      1814

Confusion matrix:
[[ 105  203]
 [ 114 1392]]
0.825248070562


### Setup naive baseline classification (hashingVectorizer)

In [10]:
vect = HashingVectorizer(ngram_range=(3,5), analyzer='char')
clf = LinearSVC()

hashing_pipeline = Pipeline([
    ('vect', vect),  # extract ngrams from tweet text
    ('clf' , clf),   # feed the output through a classifier
])
run_experiment(X, y, hashing_pipeline, "HashingVectorizer")

100%|██████████| 1/1 [00:01<00:00,  1.97s/it]

Classification Report: HashingVectorizer
               precision    recall  f1-score   support

   hatespeech       0.57      0.29      0.39       344
not_offensive       0.85      0.95      0.90      1470

  avg / total       0.80      0.82      0.80      1814

Confusion matrix:
[[ 101  243]
 [  75 1395]]
0.824696802646


### Setup tf-idf baseline classification

In [88]:
vect = TfidfVectorizer(ngram_range=(3,5), analyzer='char')
clf = LinearSVC()

tfidf_pipeline = Pipeline([
    ('vect', vect),  # extract ngrams from tweet text
    ('clf' , clf),   # feed the output through a classifier
])
run_experiment(X, y, tfidf_pipeline, "TfidfVectorizer")

100%|██████████| 1/1 [00:02<00:00,  2.46s/it]

Classification Report: TfidfVectorizer
               precision    recall  f1-score   support

   hatespeech       0.53      0.32      0.40       290
not_offensive       0.88      0.95      0.91      1524

  avg / total       0.82      0.85      0.83      1814

Confusion matrix:
[[  93  197]
 [  83 1441]]
0.845644983462


### Investigate this

In [44]:
# As much as I want to ignore this, I shouldn't. The fact that the precision score is
# close to the other experiments is troubling

X = train_test_set[['hs_keyword_count', 'comment_length', 'unknown_words_count']]
y = train_test_set['annotation_label']
clf = LinearSVC()

X_train, X_test, y_train, y_test = train_test_split(X, y)
model = clf.fit(X_train, y_train)  # train the classifier
y_prediction = model.predict(X_test)          # apply the model to the test data
report = classification_report(y_test, y_prediction)
print("Classification Report : Random Experiment")
print(report)
cm = confusion_matrix(y_test, y_prediction)
print("Confusion matrix:")
print(cm)

Classification Report : Random Experiment
               precision    recall  f1-score   support

   hatespeech       0.56      0.07      0.12       313
not_offensive       0.84      0.99      0.91      1501

  avg / total       0.79      0.83      0.77      1814

Confusion matrix:
[[  22  291]
 [  17 1484]]


### Dependency tuple experiment

In [ ]:
from itertools import chain
X = train_test_set['word_root_preRoot']
y = train_test_set['annotation_label']
clf = LinearSVC()

# http://www.markhneedham.com/blog/2015/03/02/python-scikit-learn-training-a-classifier-with-non-numeric-features/
empty_analyzer = lambda x: x
vect = DictVectorizer()
transformer = TfidfTransformer(smooth_idf=False)
# vect = TfidfVectorizer(analyzer=empty_analyzer)
# hasher = FeatureHasher(input_type='string', non_negative=True)
# X = [[str(res) for res in tmp] for tmp in X]# 
# X = hasher.transform(X)
X = X.tolist()
X = vect.fit_transform([item[0] for item in X]).toarray()
# X = transformer.fit_transform(X)
# vect.vocabulary_
X_train, X_test, y_train, y_test = train_test_split(X, y)
model = clf.fit(X_train, y_train)  # train the classifier
y_prediction = model.predict(X_test)          # apply the model to the test data
report = classification_report(y_test, y_prediction)
print("Classification Report : Random Experiment")
print(report)
cm = confusion_matrix(y_test, y_prediction)
print("Confusion matrix:")
print(cm)

### Dependency context experiment

In [82]:
from sklearn.base import TransformerMixin, BaseEstimator
class TextExtractor(BaseEstimator, TransformerMixin):
    """Adapted from code by @zacstewart 
       https://github.com/zacstewart/kaggle_seeclickfix/blob/master/estimator.py
       Also see Zac Stewart's excellent blogpost on pipelines:
       http://zacstewart.com/2014/08/05/pipelines-of-featureunions-of-pipelines.html
       """
    
    def __init__(self, column_name):
        self.column_name = column_name

    def transform(self, df):
        # select the relevant column and return it as a numpy array
        # set the array type to be string
        return df[self.column_name].tolist()
#         return np.asarray(df[self.column_name]).astype(str)
        
    def fit(self, *_):
        return self


class Apply(BaseEstimator, TransformerMixin):
    """Applies a function f element-wise to the numpy array
    """
    
    def __init__(self, fn):
        self.fn = np.vectorize(fn)
        
    def transform(self, data):
        # note: reshaping is necessary because otherwise sklearn
        # interprets 1-d array as a single sample
        return self.fn(data.reshape(data.size, 1))

    def fit(self, *_):
        return self
    
class BooleanExtractor(BaseEstimator, TransformerMixin):
    
    def __init__(self, column_name):
        self.column_name = column_name

    def transform(self, df):
        # select the relevant column and return it as a numpy array
        # set the array type to be string
        return np.asarray(df[self.column_name]).astype(np.int)
                                                       
    def fit(self, *_):
        return self

In [90]:
from itertools import chain
X = train_test_set[['feat_dependency_contexts', 'hs_keyword_count']]
y = train_test_set['annotation_label']
clf = LinearSVC()
empty_analyzer = lambda x: x
transformer = TfidfTransformer(smooth_idf=False)
vect = TfidfVectorizer(analyzer=empty_analyzer)

dependency_context_pipeline = Pipeline([
    ('dep_extractor', TextExtractor('feat_dependency_contexts')), # extract names from df
    ('vect', vect)
])

hs_keyword_count_pipeline = Pipeline([
    ('count_extractor', BooleanExtractor('hs_keyword_count')),
    ('identity', Apply(lambda x: x))
])

pipeline = Pipeline([
    ('all_features', FeatureUnion([
        ('dependency_context_pipeline', dependency_context_pipeline), # all text features
        ('hs_keyword_count_pipeline', hs_keyword_count_pipeline),
    ])),
    ('clf' , clf),   # feed the output through a classifier
])

run_experiment(X, y, pipeline, "TfidfVectorizer")

# https://github.com/michelleful/SingaporeRoadnameOrigins/blob/master/notebooks/04%20Adding%20features%20with%20Pipelines.ipynb
# https://github.com/amueller/kaggle_insults/blob/e4abac805be1d1e2b3201a978172bafd36cc01e3/features.py
# http://www.markhneedham.com/blog/2015/03/02/python-scikit-learn-training-a-classifier-with-non-numeric-features/

100%|██████████| 1/1 [00:00<00:00,  1.33it/s]

Classification Report: TfidfVectorizer
               precision    recall  f1-score   support

   hatespeech       0.54      0.30      0.38       305
not_offensive       0.87      0.95      0.91      1509

  avg / total       0.81      0.84      0.82      1814

Confusion matrix:
[[  90  215]
 [  76 1433]]
0.839581036384


### Extract dependency contexts

In [125]:
test_string = "Austrailian scientists discovers star with telescope."
doc = nlp(test_string)

# elements = []
# dict1 = {"austrailian": "scientists amod"}
# dict2 = {"scientists": "austrailian amod"}
# dict3 = {"scientists": "discovers nsubj"}

# elements.append(dict1)
# elements.append(dict2)
# elements.append(dict3)

# vect = DictVectorizer()
# X = vect.fit_transform(elements)
# X
# vect.inverse_transform(X)

def extract_dep_context(doc):
    dependency_contexts = []
    dependency_contexts_concat = []
    for word in doc:
#         if (not str(word.head.prefix_).isalpha() or not str(word.prefix_).isalpha() or "." in word.text or "." in str(word.head)):
#             pass
        if word.is_punct:
            pass
        elif len(list(word.children)) == 0:
            dependency_contexts.append({word.lower_: str(word.head.lower_) + " " + str(word.dep_) + "INV"})
            dependency_contexts_concat.append(word.lower_+ "_" + word.head.lower_ + "_" + str(word.dep_) + "INV")
        elif len(list(word.children)) >= 1:
            for child in word.children:
                if child.lower_ != word.lower_ and not child.is_punct:
                    dependency_contexts.append({word.lower_: str(child.lower_) + " " + str(child.dep_)})
                    dependency_contexts_concat.append(word.lower_ + "_" + child.lower_ + "_" + str(child.dep_))
            if word.dep_ != "ROOT":
                dependency_contexts.append({word.lower_: word.head.lower_ + " " + str(word.dep_) + "INV"})
                dependency_contexts_concat.append(word.lower_ + "_" + word.head.lower_ + "_" + str(word.dep_) + "INV")
    return dependency_contexts, dependency_contexts_concat
dependency_contexts = extract_dep_context(doc)[1]
dependency_contexts

['austrailian_scientists_amodINV',
 'scientists_austrailian_amod',
 'scientists_discovers_nsubjINV',
 'discovers_scientists_nsubj',
 'discovers_star_dobj',
 'star_with_prep',
 'star_discovers_dobjINV',
 'with_telescope_pobj',
 'with_star_prepINV',
 'telescope_with_pobjINV']

### Extract conll format

In [129]:
# https://github.com/explosion/spaCy/issues/533#issuecomment-254774296
def extract_conll_format(doc):
    result = []
    conll = []
    for sent in doc.sents:
        for i, word in enumerate(sent):
            if word.head is word:
                head_idx = 0
            else:
                 head_idx = word.head.i + 1
            conll.extend((i+1, word.lower_, word.lemma_, word.pos_, word.tag_, "_", head_idx, word.dep_, str(head_idx) + ":"+ word.dep_, "_"))
            result.append(" ".join(str(x) for x in conll))
            conll = []
    return result

conll_test = extract_conll_format(doc)
conll_test

['1 austrailian austrailian ADJ JJ _ 2 amod 2:amod _',
 '2 scientists scientist NOUN NNS _ 3 nsubj 3:nsubj _',
 '3 discovers discover VERB VBZ _ 0 ROOT 0:ROOT _',
 '4 star star NOUN NN _ 3 dobj 3:dobj _',
 '5 with with ADP IN _ 4 prep 4:prep _',
 '6 telescope telescope NOUN NN _ 5 pobj 5:pobj _']

### Create conll format file

In [ ]:
with open("hatespeech_dep_conll", "w+") as conll_file:
    for index, row in df.iterrows():
        for entry in row["conllFormat"]:
            conll_file.write(entry+"\n")
        conll_file.write("\n")
conll_file.close()